### Lab2 (a) Model preperation

In [1]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [2]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)

In [3]:
# Load the best weight paramters
net.load_state_dict(torch.load("D:\VS Projects\ECE-661\HW4\pretrained_model.pt"))
test(net)

C:\Users\18208\AppData\Local\Temp\ipykernel_24008\2190473599.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("D:\VS Projects\ECE-661\HW4\p

Files already downloaded and verified
Test Loss=0.3231, Test accuracy=0.9151


### Lab2 (b) Prune by percentage

In [27]:
import numpy as np
import torch

def prune_by_percentage(layer, q=70.0):
    """
    Prune the weights of a single layer by zeroing out weights below the q-th percentile.
    
    Args:
        layer (nn.Module): A PyTorch layer with a weight parameter.
        q (float): Percentile value (between 0 and 100). Prunes the smallest q% weights.
    """
    # 1. Convert weight to numpy array (flattened)
    weight_data = layer.weight.data.cpu().numpy()
    weight_abs_flat = np.abs(weight_data.flatten())

    # 2. Compute q-th percentile threshold
    threshold = np.percentile(weight_abs_flat, q)

    # 3. Create binary mask: keep weights >= threshold
    mask = np.where(np.abs(weight_data) >= threshold, 1.0, 0.0)

    # 4. Convert mask to torch tensor, same dtype/device as weight
    mask_tensor = torch.tensor(mask, dtype=layer.weight.dtype, device=layer.weight.device)

    # 5. Apply mask to weight
    layer.weight.data *= mask_tensor

    

In [28]:
net.load_state_dict(torch.load("pretrained_model.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # change q value
        prune_by_percentage(layer, q=70.0)
        
        # Optional: Check the sparsity you achieve in each layer
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

C:\Users\18208\AppData\Local\Temp\ipykernel_19116\2242174954.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))


Sparsity of head_conv.0.conv: 0.6990740740740741
Sparsity of body_op.0.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.0.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.1.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv1.0.conv: 0.7000868055555556
Sparsity of body_op.2.conv2.0.conv: 0.7000868055555556
Sparsity of body_op.3.conv1.0.conv: 0.6998697916666666
Sparsity of body_op.3.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.4.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.5.conv2.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv1.0.conv: 0.6999782986111112
Sparsity of body_op.6.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv1.0.conv: 0.7000054253472222
Sparsity of body_op.7.conv2.0.conv: 0.7000054253472222
Sparsity of body_op.8.conv1.0.conv: 0.7000054253472222
Sparsity of body

In [10]:
q_list = [0.2, 0.4, 0.6, 0.7, 0.8]

for q in q_list:
    # Reload original pretrained model
    net.load_state_dict(torch.load("pretrained_model.pt"))
    net = net.to(device)
    
    # Prune each layer
    for name, layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            prune_by_percentage(layer, q=100*q)  # convert to percentile (0-100)
    
    # Evaluate after pruning
    test(net)
    


C:\Users\18208\AppData\Local\Temp\ipykernel_19116\1288512156.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))


Files already downloaded and verified
Test Loss=0.3450, Test accuracy=0.9087
Files already downloaded and verified
Test Loss=0.4307, Test accuracy=0.8873
Files already downloaded and verified
Test Loss=1.0948, Test accuracy=0.7222
Files already downloaded and verified
Test Loss=2.4419, Test accuracy=0.4204
Files already downloaded and verified
Test Loss=6.3135, Test accuracy=0.1003


### Lab2 (c) Finetune pruned model

In [29]:
def finetune_after_prune(net, trainloader, criterion, optimizer, prune=True):
    """
    Finetune the pruned model for a single epoch
    Make sure pruned weights are kept as zero
    """
    # Build a dictionary for the nonzero weights
    weight_mask = {}
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Your code here: generate a mask in GPU torch tensor to have 1 for nonzero element and 0 for zero element 
            weight_mask[name] = (layer.weight.data != 0).float().to(layer.weight.device)
    
    global_steps = 0
    train_loss = 0
    correct = 0
    total = 0
    start = time.time()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        if prune:
            for name,layer in net.named_modules():
                if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                    # Your code here: Use weight_mask to make sure zero elements remains zero
                    layer.weight.data *= weight_mask[name]
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        global_steps += 1

        if global_steps % 50 == 0:
            end = time.time()
            batch_size = 256
            num_examples_per_second = 50 * batch_size / (end - start)
            print("[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                 % (global_steps, train_loss / (batch_idx + 1), (correct / total), num_examples_per_second))
            start = time.time()
    
    

In [30]:
# Get pruned model
net.load_state_dict(torch.load("pretrained_model.pt"))
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        prune_by_percentage(layer, q=80.0)

# Training setup, do not change
batch_size=256
lr=0.002
reg=1e-4

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.875, weight_decay=reg, nesterov=False)

C:\Users\18208\AppData\Local\Temp\ipykernel_19116\93151597.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [31]:
# Model finetuning
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    net.train()
    finetune_after_prune(net, trainloader, criterion, optimizer)
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if val_acc > best_acc:
        best_acc = val_acc
        print("Saving...")
        torch.save(net.state_dict(), "net_after_finetune.pt")


Epoch: 0
[Step=50]	Loss=0.9968	acc=0.6794	359.2 examples/second
[Step=100]	Loss=0.8148	acc=0.7330	24091.5 examples/second
[Step=150]	Loss=0.7257	acc=0.7602	23865.5 examples/second
Test Loss=0.5550, Test acc=0.8176
Saving...

Epoch: 1
[Step=50]	Loss=0.4674	acc=0.8443	372.9 examples/second
[Step=100]	Loss=0.4580	acc=0.8465	24669.6 examples/second
[Step=150]	Loss=0.4455	acc=0.8501	25042.1 examples/second
Test Loss=0.4879, Test acc=0.8382
Saving...

Epoch: 2
[Step=50]	Loss=0.3985	acc=0.8626	374.3 examples/second
[Step=100]	Loss=0.3948	acc=0.8654	24567.2 examples/second
[Step=150]	Loss=0.3908	acc=0.8661	24411.3 examples/second
Test Loss=0.4548, Test acc=0.8508
Saving...

Epoch: 3
[Step=50]	Loss=0.3619	acc=0.8745	374.4 examples/second
[Step=100]	Loss=0.3633	acc=0.8745	24839.2 examples/second
[Step=150]	Loss=0.3596	acc=0.8774	24825.9 examples/second
Test Loss=0.4343, Test acc=0.8563
Saving...

Epoch: 4
[Step=50]	Loss=0.3554	acc=0.8780	372.9 examples/second
[Step=100]	Loss=0.3481	acc=0.8794	2

In [32]:
# Check sparsity of the finetuned model, make sure it's not changed
net.load_state_dict(torch.load("net_after_finetune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

C:\Users\18208\AppData\Local\Temp\ipykernel_19116\3186045328.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("net_after_finetune.pt"))


Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

### Lab2 (d) Iterative pruning

In [33]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    
    net.train()
    if epoch<10:
        for name,layer in net.named_modules():
            if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                # Increase model sparsity
                q = 8.0 * (epoch + 1)
                prune_by_percentage(layer, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_iterative_prune.pt")

C:\Users\18208\AppData\Local\Temp\ipykernel_19116\1781210771.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))



Epoch: 0
[Step=50]	Loss=0.0466	acc=0.9854	368.2 examples/second
[Step=100]	Loss=0.0489	acc=0.9838	24644.7 examples/second
[Step=150]	Loss=0.0488	acc=0.9842	24703.6 examples/second
Test Loss=0.3252, Test acc=0.9153

Epoch: 1
[Step=50]	Loss=0.0459	acc=0.9859	370.0 examples/second
[Step=100]	Loss=0.0498	acc=0.9838	24622.6 examples/second
[Step=150]	Loss=0.0504	acc=0.9838	24920.3 examples/second
Test Loss=0.3259, Test acc=0.9144

Epoch: 2
[Step=50]	Loss=0.0476	acc=0.9847	372.4 examples/second
[Step=100]	Loss=0.0513	acc=0.9832	24529.7 examples/second
[Step=150]	Loss=0.0512	acc=0.9835	24829.3 examples/second
Test Loss=0.3295, Test acc=0.9136

Epoch: 3
[Step=50]	Loss=0.0540	acc=0.9828	372.8 examples/second
[Step=100]	Loss=0.0545	acc=0.9825	24552.9 examples/second
[Step=150]	Loss=0.0558	acc=0.9815	24662.3 examples/second
Test Loss=0.3320, Test acc=0.9114

Epoch: 4
[Step=50]	Loss=0.0692	acc=0.9745	370.6 examples/second
[Step=100]	Loss=0.0679	acc=0.9764	24572.0 examples/second
[Step=150]	Loss=0

In [34]:
# Check sparsity of the final model, make sure it's 80%
net.load_state_dict(torch.load("net_after_iterative_prune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

C:\Users\18208\AppData\Local\Temp\ipykernel_19116\3750828565.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("net_after_iterative_prune.pt

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

### Lab2 (e) Global iterative pruning

In [26]:
def global_prune_by_percentage(net, q=70.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # A list to gather all the weights
    flattened_weights = []
    # Find global pruning threshold
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            weight_data = layer.weight.data.cpu().numpy()
            # Flatten the weight and append to flattened_weights
            flattened_weights.append(np.abs(weight_data).flatten())
    # Concate all weights into a np array
    flattened_weights = np.concatenate(flattened_weights)
    # Find global pruning threshold
    thres = np.percentile(flattened_weights, q)
    
    # Apply pruning threshold to all layers
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            weight_data = layer.weight.data.cpu().numpy()
            # Generate a binary mask same shape as weight to decide which element to prune
            mask = np.where(np.abs(weight_data) >= thres, 1.0, 0.0)
            # Convert mask to torch tensor and put on GPU
            mask_tensor = torch.tensor(mask, dtype=layer.weight.dtype, device=layer.weight.device)
            # Multiply the weight by mask to perform pruning
            layer.weight.data *= mask_tensor

In [38]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    
    net.train()
    if epoch<10:
        # Increase model sparsity
        q = 8 * (epoch + 1)
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_global_iterative_prune.pt")

C:\Users\18208\AppData\Local\Temp\ipykernel_19116\3372515049.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))



Epoch: 0
[Step=50]	Loss=0.0452	acc=0.9861	367.0 examples/second
[Step=100]	Loss=0.0472	acc=0.9854	24700.6 examples/second
[Step=150]	Loss=0.0469	acc=0.9856	24774.2 examples/second
Test Loss=0.3239, Test acc=0.9146

Epoch: 1
[Step=50]	Loss=0.0459	acc=0.9862	368.3 examples/second
[Step=100]	Loss=0.0471	acc=0.9858	24961.9 examples/second
[Step=150]	Loss=0.0471	acc=0.9856	24956.2 examples/second
Test Loss=0.3257, Test acc=0.9153

Epoch: 2
[Step=50]	Loss=0.0485	acc=0.9845	370.6 examples/second
[Step=100]	Loss=0.0515	acc=0.9831	24807.3 examples/second
[Step=150]	Loss=0.0507	acc=0.9836	24823.4 examples/second
Test Loss=0.3270, Test acc=0.9152

Epoch: 3
[Step=50]	Loss=0.0512	acc=0.9828	346.0 examples/second
[Step=100]	Loss=0.0512	acc=0.9836	24825.6 examples/second
[Step=150]	Loss=0.0526	acc=0.9829	24644.4 examples/second
Test Loss=0.3275, Test acc=0.9130

Epoch: 4
[Step=50]	Loss=0.0582	acc=0.9797	369.0 examples/second
[Step=100]	Loss=0.0580	acc=0.9801	24713.6 examples/second
[Step=150]	Loss=0

In [39]:
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

C:\Users\18208\AppData\Local\Temp\ipykernel_19116\624437447.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("net_after_global_iterative_pr

Sparsity of head_conv.0.conv: 0.3101851851851852
Sparsity of body_op.0.conv1.0.conv: 0.6571180555555556
Sparsity of body_op.0.conv2.0.conv: 0.6393229166666666
Sparsity of body_op.1.conv1.0.conv: 0.6271701388888888
Sparsity of body_op.1.conv2.0.conv: 0.6471354166666666
Sparsity of body_op.2.conv1.0.conv: 0.6319444444444444
Sparsity of body_op.2.conv2.0.conv: 0.6692708333333334
Sparsity of body_op.3.conv1.0.conv: 0.6236979166666666
Sparsity of body_op.3.conv2.0.conv: 0.6884765625
Sparsity of body_op.4.conv1.0.conv: 0.7254774305555556
Sparsity of body_op.4.conv2.0.conv: 0.7825520833333334
Sparsity of body_op.5.conv1.0.conv: 0.7240668402777778
Sparsity of body_op.5.conv2.0.conv: 0.8133680555555556
Sparsity of body_op.6.conv1.0.conv: 0.7327473958333334
Sparsity of body_op.6.conv2.0.conv: 0.7648111979166666
Sparsity of body_op.7.conv1.0.conv: 0.7766927083333334
Sparsity of body_op.7.conv2.0.conv: 0.8260633680555556
Sparsity of body_op.8.conv1.0.conv: 0.8528103298611112
Sparsity of body_op.8.

### Lab 3 (b) and (c): Fixed-point quantization

In [ ]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

C:\Users\18208\AppData\Local\Temp\ipykernel_13636\3364891440.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))


Files already downloaded and verified
Test Loss=0.3231, Test accuracy=0.9151


In [12]:
# Define quantized model and load weight
Nbits = 2 #Change this value to finish (b) and (c)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)


C:\Users\18208\AppData\Local\Temp\ipykernel_24008\976017615.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))


Files already downloaded and verified
Test Loss=9.6141, Test accuracy=0.0899


In [23]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)   

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=1.3454	acc=0.6265	318.4 examples/second
[Step=100]	Loss=1.0711	acc=0.6881	6645.3 examples/second
[Step=150]	Loss=0.9381	acc=0.7182	7106.4 examples/second
Test Loss=0.7360, Test acc=0.7789
Saving...

Epoch: 1
[Step=200]	Loss=0.6473	acc=0.7959	226.2 examples/second
[Step=250]	Loss=0.5767	acc=0.8070	3920.8 examples/second
[Step=300]	Loss=0.5652	acc=0.8091	5836.3 examples/second
[Step=350]	Loss=0.5481	acc=0.8148	5425.8 examples/second
Test Loss=0.6448, Test acc=0.8053
Saving...

Epoch: 2
[Step=400]	Loss=0.5375	acc=0.8198	253.5 examples/second
[Step=450]	Loss=0.4879	acc=0.8373	8672.3 examples/second
[Step=500]	Loss=0.4820	acc=0.8360	7989.3 examples/second
[Step=550]	Loss=0.4680	acc=0.8397	8554.0 examples/second
Test Loss=0.6166, Test acc=0.8129
Saving...

Epoch: 3
[Step=600]	Loss=0.4285	acc=0.8558	245.7 examples/second
[Step=650]	Loss=0.4324	acc=0.8510	8361.4 examples/se

C:\Users\18208\AppData\Local\Temp\ipykernel_13636\4294047440.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("quantized_net_after_finetune

Files already downloaded and verified
Test Loss=0.4430, Test accuracy=0.8592


### Lab3 (d) Quantize pruned model

In [31]:
# Define quantized model and load weight
Nbits = 2 #Change this value to finish (d)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net)


C:\Users\18208\AppData\Local\Temp\ipykernel_13636\2936244215.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("net_after_global_iterative_p

Files already downloaded and verified
Test Loss=1730.1061, Test accuracy=0.1000


In [32]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=2.3332	acc=0.1305	386.6 examples/second
[Step=100]	Loss=2.2791	acc=0.1463	19899.4 examples/second
[Step=150]	Loss=2.2524	acc=0.1600	19643.5 examples/second
Test Loss=2.2056, Test acc=0.2014
Saving...

Epoch: 1
[Step=200]	Loss=2.1662	acc=0.1885	336.0 examples/second
[Step=250]	Loss=2.1485	acc=0.2067	19632.7 examples/second
[Step=300]	Loss=2.1424	acc=0.2142	20012.4 examples/second
[Step=350]	Loss=2.1347	acc=0.2178	19551.9 examples/second
Test Loss=2.1146, Test acc=0.2303
Saving...

Epoch: 2
[Step=400]	Loss=2.0758	acc=0.2402	334.3 examples/second
[Step=450]	Loss=2.0907	acc=0.2329	19376.9 examples/second
[Step=500]	Loss=2.0880	acc=0.2337	19148.1 examples/second
[Step=550]	Loss=2.0871	acc=0.2350	19515.7 examples/second
Test Loss=2.1415, Test acc=0.2370
Saving...

Epoch: 3
[Step=600]	Loss=2.0875	acc=0.2507	336.4 examples/second
[Step=650]	Loss=2.0733	acc=0.2445	19958.5 ex

C:\Users\18208\AppData\Local\Temp\ipykernel_13636\2253028306.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("quantized_net_after_finetune

Files already downloaded and verified
Test Loss=1.7723, Test accuracy=0.3264


Nbits = 6 Test Loss=0.3365, Test accuracy=0.9144
Nbits = 5 Test Loss=0.3391, Test accuracy=0.9113
Nbits = 4 Test Loss=0.3862, Test accuracy=0.8973
Nbits = 3 Test Loss=0.9869, Test accuracy=0.7660
Nbits = 2 Test Loss=9.6141, Test accuracy=0.0899

### Lab4 (a) Symmetric quantization
#### Implement symmetric quantization in FP_layers.py, and repeat the process in (b)

In [10]:
# check the performance of symmetric quantization with 6, 5, 4, 3, 2 bits
Nbits = 6 #Change this value to finish (b) and (c)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits, symmetric=True)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

C:\Users\18208\AppData\Local\Temp\ipykernel_24008\4165186008.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load("pretrained_model.pt"))


Files already downloaded and verified
Test Loss=0.3275, Test accuracy=0.9134
